# Homework 1

Implementations of Travelling Salesman and course scheduling problems for CS-344 by Professor Keith VanderLinden, Calvin University.

Completed by Nathan Meyer, 2/29/2020.

## 1.1

Introspection might offer some uniquely intimate insights into reasoning, emotions, and responses of individuals. In that way, it might offer some insight into how that might be modeled in an AI. At the same time, as a part of one's own perception, it might be difficult to ascertain reliable and quantifiable details that could translate to a computing model.

Furthermore, the reliability and accuracy of an individual's introspection could be problematic. What an individual perceives to be their mental processes may not be entirely accurate to what is really happening, and their perceptions could vary wildly from individual to individual. So modeling an AI on one person's perceptions of their mental processes might not only be inaccurate to how these processes really work, they might also be inconsistent with the processes of other individuals. 

## 1.2

### Formulation
Local search implementation of Travelling Salesman Problem based on these foundations:
- States: Complete sequences of visiting each city and returning to the first, e.g. [0,1,2,3,4,0]
- Actions: Swapping two cities in sequence of index i and j, e.g. [[1,2],[2,3],[3,4],...]
- Result (transition): New state with cities swapped based on given action
- Values (heuristics): Distance between two cities i and j


In [2]:
from search import Problem

class TSP(Problem):
    """
    State: sequence of visited cities
    Actions: swap two cities in sequence of index i and j
    Result: new sequence of sequences after action
    Value: distance between two cities i and j
    """

    def __init__(self, initial, distances):
        self.initial = initial
        self.distances = distances

    def actions(self, state):
        """From a given state, return possible actions (swaps).
        Actions include swapping any cities in the sequence between the
        first and the last.
        """
        actions = []

        for i in range(1, len(state)-1):
            for j in range(1, len(state)-1):
                if i != j:
                    actions.append((i, j))

        return actions
    
    def result(self, state, action):
        """With given action, swap the cities and return resulting state"""
        new_state = state[:]

        # Gather cities from state at indeces given by action
        swap1 = state[action[0]]
        swap2 = state[action[1]]
        # Swap those in new_state
        new_state[action[0]] = swap2
        new_state[action[1]] = swap1

        return new_state

    def value(self, state):
        """Computes and returns total distance travelled
        as negative value, to switch local maximum to minimum"""
        value = 0
        for i in range(len(state)-1):
            city1 = state[i]
            city2 = state[i+1]
            # Frozenset to evaluate distance regardless of order
            value -= self.distances[frozenset((city1, city2))]

        return value

### Execution
With a set number of cities and set maximum distance between them, the following code generates a random sequence of numerical cities with a random starting/ending point (since it is a looping path). This is then fed into the hill-climbing and simulated-annealing algorithms.

In [6]:
from search import hill_climbing, simulated_annealing, exp_schedule
from random import randrange, shuffle
import time

cities = 8
max_distance = 20

# Generate initial sequence of cities
initial = []
for city in range(1, cities+1):
    initial.append(city)
shuffle(initial)
# Make the initial state a loop by adding first city to end
initial.append(initial[0])

# Generate random distances between each city
distances = {}

for city1 in initial:
    for city2 in initial:
        if city1 != city2:  # frozensets to ensure no duplicates
            distances[frozenset((city1, city2))] = randrange(1, max_distance)

# Generate and print initial TSP problem
problem = TSP(initial, distances)
print('Initial:\n\t' + str(problem.initial)
        + "\tvalue: " + str(-problem.value(initial))
    )

# Solve the problem with hill-climbing
time1 = time.time()
hill_solution = hill_climbing(problem)
time2 = time.time()
print('Hill-climbing solution:\n\t' + str(hill_solution)
        + '\tvalue: ' + str(-problem.value(hill_solution))
        + "\t\ttime: %0.3f seconds" % (time2 - time1)
    )

# Solve the problem with simulated annealing
time1 = time.time()
annealing_solution = simulated_annealing(
    problem,
    exp_schedule(k=20, lam=0.05, limit=1000)
)
time2 = time.time()
print('Simulated annealing solution:\n\t' + str(annealing_solution)
        + '\tvalue: ' + str(-problem.value(annealing_solution))
        + "\t\ttime: %0.3f seconds" % (time2 - time1)
    )

Initial:
	[8, 2, 7, 3, 1, 4, 5, 6, 8]	value: 70
Hill-climbing solution:
	[8, 5, 2, 1, 3, 6, 7, 4, 8]	value: 25		time: 0.002 seconds
Simulated annealing solution:
	[8, 7, 4, 5, 2, 1, 3, 6, 8]	value: 26		time: 0.102 seconds


### Reflection
The results between hill-climbing and simulated annealing are comparable. With randomly generating the distances between each city, sometimes hill-climbing will produce a lower overall distance value, and sometimes simulated annealing will. The difference is rarely greater than about 5. As for the times, hill-climbing is much faster: it usually takes 0.002-0.004 seconds, while simulated annealing takes anywhere from 0.095-0.120 seconds.

This makes sense, given that hill-climbing is simply a loop to find the "best" neighboring state; in this case, it's simply the value closest to 0 (due to producing negative values). The algorithm is not computationally intensive, so it is unsurprising that is considerably faster than simulated annealing. Given that simulated annealing is inherently more complex, the slower time also makes sense. However, it is curious that hill-climbing will occasionally produce equivalent or *better* results than simulated annealing. Perhaps this is because the problem is set up such that it is relatively "easy" for both algorithms to find a similarly suitable solution. However, in the cases of when hill-climbing produces a worse solution, it is easy to imagine that it gets stuck, given its simplicity. At the same time, since the problem is NP-hard, it can be possible that something like simulated annealing cannot properly solve for the best value, thereby allowing hill-climbing to occasionally perform better. 

Finally, both were pretty slow on larger sets of cities. For 100 cities, hill-climbing took 27 seconds to find a solution, and simulated-annealing took just under 60 seconds.

## 1.3

### Formulation
Constraint satisfaction implementation to solve how to schedule courses using this formulation:
- Variables: Courses
- Domains: Tuples of Prof + Time + Room
- Constraints:
    - One Prof assignment per course
    - Each course offered only once
    - Prof can only teach one course at a time
    - No double booked rooms

In [ ]:
from csp import CSP, parse_neighbors

def CourseScheduling():
    """Return an example of constraint satisfaction for course scheduling"""
    Faculty = 'VanderLinden Norman Adams Plantinga'.split()
    Rooms = 'NH253 SB382'.split()
    Times = 'MWF900 MWF1200 TTH1030 TTH1200'.split()
    variables = 'CS108 CS112 CS212 CS214 CS232 CS262'.split()

    # Create all possible domain values for each variable (course)
    # arrangement is always tuples of prof, time, and room
    domains = {}
    for course in variables:
        domains[course] = []
        for prof in Faculty:
            for time in Times:
                for room in Rooms:
                    domains[course].append((prof, time, room))

    # Make sure each course is a neighbor
    neighbors = parse_neighbors("""CS108: CS112 CS212 CS214 CS232 CS262
                CS112: CS212 CS214 CS232 CS262
                CS212: CS214 CS232 CS262
                CS214: CS232 CS262
                CS232: CS262""", variables)

    def course_constraints(A, a, B, b):
        """Defines constraints for scheduling with variable courses A and B
        and their respective values a and b
        Returns true only if none of the constraints are violated"""

        # Are the assignments correct?
        if (A == 'CS108' and a[0] != 'VanderLinden') or \
           (B == 'CS108' and b[0] != 'VanderLinden'):
            return False
        elif (A == 'CS112' and a[0] != 'Norman') or \
             (B == 'CS112' and b[0] != 'Norman'):
            return False
        elif (A == 'CS212' and a[0] != 'Plantinga') or \
             (A == 'CS212' and b[0] != 'Plantinga'):
            return False
        elif (A == 'CS214' and a[0] != 'Adams') or \
             (B == 'CS214' and b[0] != 'Adams'):
            return False
        elif (A == 'CS232' and a[0] != 'Norman') or \
             (B == 'CS232' and b[0] != 'Norman'):
            return False
        elif (A == 'CS262' and a[0] != 'VanderLinden') or \
             (B == 'CS262' and b[0] != 'VanderLinden'):
            return False
        # Is the teacher assigned twice to one time?
        elif a[0] == b[0] and a[1] == a[2]:
            return False
        # Is the room double-booked?
        elif a[1] == b[1] and a[2] == b[2]:
            return False
        # Else, constraints are satisfied
        return True
    return CSP(variables, domains, neighbors, course_constraints)